In [1]:
import tensorflow as tf

In [2]:
a = tf.Variable(2)   #创建标量张量
b = tf.Variable([2])   #创建向量张量
c = tf.Variable([[2]])   #创建矩阵张量
print(a)
print(b)
print(c)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=2>
<tf.Variable 'Variable:0' shape=(1,) dtype=int32, numpy=array([2])>
<tf.Variable 'Variable:0' shape=(1, 1) dtype=int32, numpy=array([[2]])>


In [3]:
a = tf.Variable(2)
b = tf.Variable(3,trainable=False)
print(a.trainable)   #查看张量状态
print(b.trainable) 

True
False


In [4]:
a = tf.Variable(2)
print(a.assign(3))   #赋予一个整型值

<tf.Variable 'UnreadVariable' shape=() dtype=int32, numpy=3>


In [5]:
a = tf.Variable(2)
print(a.assign('3'))   #赋予一个字符串值

TypeError: Cannot convert '3' to EagerTensor of dtype int32

In [6]:
x = tf.constant(3.0)
with tf.GradientTape() as tape:   #梯度求导
    tape.watch(x)   #常量张量跟踪
    y = x*x*x
dy_dx = tape.gradient(y,x)
print(dy_dx)

tf.Tensor(27.0, shape=(), dtype=float32)


In [7]:
x = tf.Variable(3.0)
with tf.GradientTape() as tape:
    y = x*x*x
dy_dx = tape.gradient(y,x)
print(dy_dx)

tf.Tensor(27.0, shape=(), dtype=float32)


In [8]:
x = tf.Variable(3.0,trainable=False)
with tf.GradientTape() as tape:
    y = x*x*x
dy_dx = tape.gradient(y,x)
print(dy_dx)

None


In [9]:
x = tf.Variable(3.0,trainable=False)
with tf.GradientTape() as tape:
    y = x*x*x
dy_dx = tape.gradient(y,x)
d2y_d2x = tape.gradient(dy_dx,x)
print(d2y_d2x)

RuntimeError: A non-persistent GradientTape can only be used tocompute one set of gradients (or jacobians)

In [10]:
x = tf.Variable(3.0)
with tf.GradientTape() as tape:   #二次梯度求导
    with tf.GradientTape() as tape2:
        y = x*x*x
    dy_dx = tape2.gradient(y,x)
d2y_d2x = tape.gradient(dy_dx,x)
print(dy_dx)
print(d2y_d2x)

tf.Tensor(27.0, shape=(), dtype=float32)
tf.Tensor(18.0, shape=(), dtype=float32)


In [11]:
x = tf.Variable(2.0)
with tf.GradientTape(persistent=True) as tape:   #多运算式梯度求导
    y = x*x*x
    z = x*x
dy_dx = tape.gradient(y,x)
dz_dx = tape.gradient(z,x)
print(dy_dx)
print(dz_dx)
del tape   #删除梯度带，释放资源

tf.Tensor(12.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [12]:
x = tf.Variable(2.0)
k = tf.Variable(3.0)
with tf.GradientTape(persistent=True,watch_accessed_variables=False) as tape:
    tape.watch(x)
    y = x*x*x
    z = k*k
dy_dx = tape.gradient(y,x)
dz_dx = tape.gradient(z,x)
print(dy_dx)
print(dz_dx)
del tape

tf.Tensor(12.0, shape=(), dtype=float32)
None


In [13]:
@tf.function   #将python函数转为静态图执行
def add(x): 
    return x + 1 
isinstance(add.get_concrete_function(1).graph, tf.Graph)   #判断是否为图

True

In [14]:
def add(x): 
    return x + 1 
isinstance(add.get_concrete_function(1).graph, tf.Graph)

AttributeError: 'function' object has no attribute 'get_concrete_function'

In [15]:
@tf.function 
def f(x): 
    if x > 0: 
        return x 
    else: 
        return -x 
f(tf.constant(-2))

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [16]:
array = []
@tf.function 
def f(x): 
    for i in range(len(x)):
        array.append(x[i])
    print(array)
f(tf.constant([1, 2, 3]))

[<tf.Tensor 'strided_slice:0' shape=() dtype=int32>, <tf.Tensor 'strided_slice_1:0' shape=() dtype=int32>, <tf.Tensor 'strided_slice_2:0' shape=() dtype=int32>]


In [17]:
@tf.function 
def f(x):
    array = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True) 
    for i in range(len(x)):
        array = array.write(i,x[i])
    tf.print(array.stack())
f(tf.constant([1, 2, 3]))

[1 2 3]


In [18]:
@tf.function
def f(x):
    print(x)
    return x + x
print(f(tf.constant(2)))
print(f(tf.constant(2.0)))
print(f(tf.constant('2.0')))

Tensor("x:0", shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
Tensor("x:0", shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)
Tensor("x:0", shape=(), dtype=string)
tf.Tensor(b'2.02.0', shape=(), dtype=string)


In [19]:
@tf.function
def f(x):
    return x + x
f1 = f.get_concrete_function(1)   #得到当前变量函数返回的类型
f2 = f.get_concrete_function(2)
print(f1 is f2)   #判断二者类型是否相同

False


In [20]:
@tf.function
def f(x):
    return x + x
f1 = f.get_concrete_function(tf.constant(1)) 
f2 = f.get_concrete_function(tf.constant(2))
print(f1 is f2)

True
